# Notebook for loading in Parse Bio AnnData object and spliting into two separate objects for each study

In [1]:
# # These packages are pre-installed on Google Colab, but are included here to facilitate running this notebook locally
# !pip install --quiet matplotlib
# !pip install --quiet scikit-learn
# !pip install --quiet numpy
# !pip install --quiet scipy

In [2]:
# %%time
# # Install scanpy and other packages needed for single-cell RNA-seq analysis
# !pip install --quiet scanpy python-igraph louvain pybiomart #MulticoreTSNE

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
docker 6.1.3 requires websocket-client>=0.32.0, which is not installed.
CPU times: user 599 ms, sys: 80.8 ms, total: 679 ms
Wall time: 30.8 s


In [1]:
# Import packages
import anndata
import matplotlib
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.decomposition import TruncatedSVD
from scipy import sparse, io
from tqdm import tqdm

matplotlib.rcParams.update({'font.size': 12})
%config InlineBackend.figure_format = 'retina'

Loading AnnData Object

In [2]:
data_path = "/resnick/groups/mthomson/jboktor/split-pipe_workflow/combined_parallel/all-sample/DGE_filtered"
adata_path = os.path.join(data_path, 'anndata.h5ad')
analysis_path = "/central/groups/MazmanianLab/joeB/snRNAseq_analysis/WILDRxSPF_brains"
output_path = os.path.join(analysis_path, "data/input/parse_bio_anndata")
os.makedirs(output_path, exist_ok=True)

In [3]:
adata = anndata.read_h5ad(adata_path)
adata

AnnData object with n_obs × n_vars = 719168 × 78298
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count'
    var: 'gene_id', 'gene_name', 'genome'

In [4]:
adata.var_names.tolist()
# adata.var_names = [name.replace('_GRCm39', '') for name in adata.var_names]
adata.var_names = [name.split('_GRCm39')[0] for name in adata.var_names]
adata.var_names

Index(['Gnai3', 'Pbsn', 'Cdc45', 'H19', 'Scml2', 'Apoh', 'Narf', 'Cav2',
       'Klf6', 'Scmh1',
       ...
       'Gm55949', 'Gm56260', 'Gm56355', 'Gm54430', 'Gm54490', 'Gm56371',
       'Gm23510', 'Gm22711', 'Gm55627', 'Gm54807'],
      dtype='object', length=78298)

In [8]:
# Load the gene mapping file
gene_mapping = pd.read_csv("/resnick/groups/mthomson/jboktor/split-pipe_workflow/combined_parallel/all-sample/DGE_filtered/all_genes.csv")
gene_mapping

# Check for duplicates in gene_id
n_gene_ids = len(gene_mapping['gene_id'])
n_unique_gene_ids = len(gene_mapping['gene_id'].unique())
print(f"\nGene IDs:")
print(f"Total count: {n_gene_ids}")
print(f"Unique count: {n_unique_gene_ids}")
print(f"Number of duplicates: {n_gene_ids - n_unique_gene_ids}")

if n_gene_ids != n_unique_gene_ids:
    print("\nDuplicate gene_ids:")
    print(gene_mapping[gene_mapping['gene_id'].duplicated(keep=False)].sort_values('gene_id'))

# Check for duplicates in gene_name
n_gene_names = len(gene_mapping['gene_name'])
n_unique_gene_names = len(gene_mapping['gene_name'].unique())
print(f"\nGene Names:")
print(f"Total count: {n_gene_names}")
print(f"Unique count: {n_unique_gene_names}")
print(f"Number of duplicates: {n_gene_names - n_unique_gene_names}")

if n_gene_names != n_unique_gene_names:
    print("\nDuplicate gene_names:")
    print(gene_mapping[gene_mapping['gene_name'].duplicated(keep=False)].sort_values('gene_name'))


Gene IDs:
Total count: 78298
Unique count: 78298
Number of duplicates: 0

Gene Names:
Total count: 78298
Unique count: 77981
Number of duplicates: 317

Duplicate gene_names:
                  gene_id      gene_name  genome
54335  ENSMUSG00000121691  1600029O15Rik  GRCm39
16406  ENSMUSG00000057818  1600029O15Rik  GRCm39
56229  ENSMUSG00000123653  1700016A09Rik  GRCm39
37421  ENSMUSG00000102737  1700016A09Rik  GRCm39
54513  ENSMUSG00000121875  1700020L13Rik  GRCm39
...                   ...            ...     ...
54151  ENSMUSG00000121502         Zfp939  GRCm39
42083  ENSMUSG00000108043           Zim3  GRCm39
54516  ENSMUSG00000121878           Zim3  GRCm39
47252  ENSMUSG00000113593         Zp4-ps  GRCm39
54026  ENSMUSG00000121372         Zp4-ps  GRCm39

[629 rows x 3 columns]


All Ensemble ID's are unique but some gene_names are duplicates - must be mapped to different ensemble IDs

Swapping in gene symbol names for OG Ensembl IDs

In [15]:
adata.var_names = gene_mapping['gene_id']

n_total = len(adata.var_names)
n_unique = len(set(adata.var_names))

print(f"Total number of names: {n_total}")
print(f"Number of unique names: {n_unique}")
print(f"Are all names unique? {n_total == n_unique}")

# If there are duplicates, we can find them:
if n_total != n_unique:
    duplicates = pd.Series(adata.var_names)[pd.Series(adata.var_names).duplicated()].unique()
    print("\nDuplicate names found:")
    print(duplicates)
    print(f"\nNumber of duplicate names: {len(duplicates)}")

Total number of names: 78298
Number of unique names: 78298
Are all names unique? True


Saving AnnData with unique cellxgene matrix with unique ensemble IDs for Cell Type Mapping

In [21]:
print(os.path.join(output_path, 'anndata_ensembl.h5ad'))

/central/groups/MazmanianLab/joeB/snRNAseq_analysis/WILDRxSPF_brains/data/input/parse_bio_anndata/anndata_ensembl.h5ad


In [20]:
print(os.path.join(output_path, 'anndata_ensembl.h5ad'))
# Saving the AnnData object with Ensembl IDs for Cell Type Mapping
adata.write_h5ad(os.path.join(output_path, 'anndata_ensembl.h5ad'))

# Filtering samples for each study

In [ ]:
# Clean load of data
adata = anndata.read_h5ad(adata_path)
adata

In [ ]:
# 2. Filter cells
sc.pp.filter_cells(adata, min_counts=1000, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3, min_counts=10)

In [10]:
manxuan_adata  = adata[adata.obs['sample'].str.contains('ASO|WT', regex=True)]
print(manxuan_adata)
for sample in manxuan_adata.obs['sample'].unique():
    print(sample)

sc.pp.filter_cells(manxuan_adata, min_counts=1000)  # keep cells with ≥1,000 total counts :contentReference[oaicite:0]{index=0}
sc.pp.filter_cells(manxuan_adata, min_genes=200)    # keep cells with ≥200 genes detected :contentReference[oaicite:1]{index=1}
sc.pp.filter_genes(manxuan_adata, min_cells=20)      # keep genes seen in ≥20 cells :contentReference[oaicite:2]{index=2}
sc.pp.filter_genes(manxuan_adata, min_counts=10)    # keep genes with ≥10 total counts :contentReference[oaicite:3]{index=3}

print(manxuan_adata)
manxuan_adata.write_h5ad(os.path.join(output_path, 'aso_wt_data.h5ad'))

View of AnnData object with n_obs × n_vars = 301709 × 78298
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count'
    var: 'gene_id', 'gene_name', 'genome'
ASO11
ASO13
ASO15
ASO16
ASO17
ASO20
WT11
WT12
WT13
WT14
WT15
WT17


/central/groups/MazmanianLab/joeB/software/mambaforge/envs/anndata/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:168: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_counts"] = number


AnnData object with n_obs × n_vars = 300016 × 51948
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'n_counts', 'n_genes'
    var: 'gene_id', 'gene_name', 'genome', 'n_cells', 'n_counts'


In [12]:
jess_adata = adata[~adata.obs['sample'].str.contains('ASO|WT', regex=True)]
print(jess_adata)
for sample in jess_adata.obs['sample'].unique():
    print(sample)
    
sc.pp.filter_cells(jess_adata, min_counts=1000)  # keep cells with ≥1,000 total counts
sc.pp.filter_cells(jess_adata, min_genes=200)    # keep cells with ≥200 genes detected
sc.pp.filter_genes(jess_adata, min_cells=20)      # keep genes seen in ≥20 cells
sc.pp.filter_genes(jess_adata, min_counts=10)    # keep genes with ≥10 total counts

print(jess_adata)
jess_adata.write_h5ad(os.path.join(output_path, 'wildrxspf_data.h5ad'))

View of AnnData object with n_obs × n_vars = 417459 × 78298
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count'
    var: 'gene_id', 'gene_name', 'genome'
SPF_10_HYP
SPF_11_AMY
SPF_1D4_HYP
SPF_3D4_AMY
SPF_3D4_HYP
SPF_6_AMY
SPF_6_HYP
SPF_7_AMY
SPF_8_AMY
SPF_8_HYP
SPF_9_AMY
SPF_9_HYP
WildR_10_AMY
WildR_10_HYP
WildR_11_AMY
WildR_11_HYP
WildR_3D4_HYP
WildR_5_AMY
WildR_6_AMY
WildR_6_HYP
WildR_7_AMY
WildR_7_HYP
WildR_9_AMY
WildR_9_HYP
AnnData object with n_obs × n_vars = 414591 × 56721
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'n_counts', 'n_genes'
    var: 'gene_id', 'gene_name', 'genome', 'n_cells', 'n_counts'
